In [ ]:
import os
import numpy as np
from os.path import expanduser
from fcs_signal_prediction.main import main

%load_ext autoreload
%reload_ext autoreload
%autoreload 2

In [ ]:
DATA_CONVERGE_PROJECT="sd2e-project-43"

data_converge_base = os.path.join(expanduser("~"), 'sd2e-projects', DATA_CONVERGE_PROJECT)
experiment_dir = os.path.realpath(os.path.join(data_converge_base, 'reactor_outputs', 'complete'))
experiment_dir_contents = [os.path.realpath(os.path.join(experiment_dir, x)) for x in os.listdir(experiment_dir)]

experiment_id="YeastSTATES-OR-Gate-CRISPR-Dose-Response"

process_dir = os.path.join(experiment_dir, experiment_id)
experiment_id_dir_contents = [os.path.realpath(os.path.join(process_dir, x)) for x in os.listdir(process_dir)]
experiment_id_dir_contents.sort()

last_process = experiment_id_dir_contents[-1]

In [ ]:
high_control = 'CRISPR_CEN.PK2_positive_control_NOR_00_24864'
low_control = 'CRISPR_CEN.PK2_negative_control_23970'

result, timeseries_fig, samples_and_controls_fig = main(last_process,
              experiment_id,
              low_control,
              high_control)